In [1]:
from gensim.models import Word2Vec
import gensim.downloader as api
import numpy as np
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import f1_score

In [2]:
v2w_model = api.load('word2vec-google-news-300')
sample_word2vec_embedding = v2w_model['computer'];

glove_model = api.load('glove-twitter-25')
sample_glove_embedding = glove_model['computer'];

In [3]:
embedding1 = v2w_model
embedding2 = glove_model

In [4]:
def process_word(line):
    word, tag = line.split('\t')
    tag = tag[:-1]
    try:
        emb_word1 = np.append(embedding1[word], np.zeros(1))
    except:
        emb_word1 = np.append(np.zeros(300, dtype="float32"), 100000*np.ones(1))
    try:
        emb_word2 = np.append(embedding2[word], np.zeros(1))
    except:
        emb_word2 = np.append(np.zeros(25, dtype="float32"), 100000*np.ones(1))
    emb_word = np.append(emb_word1, emb_word2)
    tag = 1 if tag != 'O' else 0
    return emb_word, tag

In [5]:
def load_data(path):
    with open(path, 'r', encoding='utf-8') as file:
        sentences = []
        labels = []
        curr_sen = []
        curr_labels = []
        for line in file.readlines():
            if len(line.split('\t')) > 1  and line != '\t' and line != '\n' and len(line.split('\t')[-1]) > 1:
                emb_word, tag = process_word(line)
                curr_sen.append(emb_word)
                curr_labels.append(tag)
            else:
                sentences.append(curr_sen[::])
                labels.append(curr_labels[::])
                curr_sen = []
                curr_labels = []  
        return sentences, labels

In [6]:
train_sentences, train_labels = load_data("data/train.tagged")

In [7]:
dev_sentences, dev_labels = load_data("data/dev.tagged")

In [8]:
def contex_embeddings(sentences):
    contex_embs = []
    for sent in sentences:
        for i in range(len(sent)):
            if i == 0:
                if len(sent) == 1:
                    contex_emb = np.concatenate((sent[i], sent[i], sent[i]))
                elif len(sent) == 2:
                    contex_emb = np.concatenate((sent[i], sent[i], sent[i+1]))
                else:
                    contex_emb = np.concatenate((sent[i], sent[i+1], sent[i+2]))
            elif i == len(sent) - 1:
                if len(sent) == 2:
                    contex_emb = np.concatenate((sent[i], sent[i], sent[i-1]))
                else:
                    contex_emb = np.concatenate((sent[i], sent[i-1], sent[i-2]))
            else:
                contex_emb = np.concatenate((sent[i], sent[i-1], sent[i+1]))
            contex_embs.append(contex_emb)
    return np.array(contex_embs)

In [9]:
train_labels_flat = [item for sublist in train_labels for item in sublist]
dev_labels_flat = [item for sublist in dev_labels for item in sublist]

In [10]:
train_sentences_flat = [item for sublist in train_sentences for item in sublist]
dev_sentences_flat = [item for sublist in dev_sentences for item in sublist]

In [11]:
model_knn = KNeighborsClassifier(n_neighbors=1)
model_knn.fit(train_sentences_flat, train_labels_flat)

KNeighborsClassifier(n_neighbors=1)

In [12]:
predict_dev_knn = model_knn.predict(dev_sentences_flat)
f1_score(dev_labels_flat, predict_dev_knn)

0.5831622176591376

## Part 2

In [13]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch

In [14]:
class TweetsDataset(Dataset):

    def __init__(self, embeddings_flat, labels_flat):
        self.embeddings = embeddings_flat
        self.labels = labels_flat

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return [self.embeddings[idx], self.labels[idx]]

In [15]:
train_emb = torch.tensor(np.array(train_sentences_flat)).float()
dev_emb = torch.tensor(np.array(dev_sentences_flat)).float()
train_labels = torch.tensor(np.array(train_labels_flat))
dev_labels = torch.tensor(np.array(dev_labels_flat))

In [16]:
train_dataset = TweetsDataset(train_emb, train_labels)
dev_dataset = TweetsDataset(dev_emb, dev_labels)

In [17]:
batch_size = 64

In [18]:
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
dev_loader = DataLoader(dev_dataset, batch_size = batch_size, shuffle = False)

In [19]:
device = "cpu"

In [20]:
class SentimentNN(nn.Module):

    def __init__(self, input_dim=327):
        super(SentimentNN, self).__init__()
        self.first_layer = nn.Linear(input_dim, 200)
        self.second_layer = nn.Linear(200, 100)
        self.third_layer = nn.Linear(100, 1)

    def forward(self, input_ids, labels=None):
        x = self.first_layer(input_ids)
        x = nn.ReLU()(x)
        x = self.second_layer(x)
        x = nn.Tanh()(x)
        x = self.third_layer(x)
        return x

In [21]:
def train_and_test(epochs, train_loader, test_loader):
    our_net = SentimentNN()
    
    if torch.cuda.is_available():
        our_net = our_net.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(our_net.parameters(), lr=0.001)
    for epoch in range(epochs):
        our_net.train()
        for i, (sents, labels) in enumerate(train_loader):
            if torch.cuda.is_available():
                sents = sents.to(device)
                #labels = torch.cuda.LongTensor(labels)
                labels = labels.float().to(device)

            # Forward + Backward + Optimize
            outputs = our_net(sents).view(-1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            if (i+1) % 300 == 0:
                print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                       %(epoch+1, epochs, i+1,
                         0.8 * len(train_dataset)//batch_size, loss.data))
        our_net.eval()        
        acc = accuracy_calc(test_loader, our_net)
        print(acc)
    return accuracy_calc(test_loader, our_net)

In [22]:
def get_accuracy(y_true, y_preds):
    return (y_true == y_preds).sum().item() / y_true.size(0)

def accuracy_calc(loader, net):
    i = sum_accuracy = 0
    sum_loss = 0
    criterion = nn.BCEWithLogitsLoss()
    # Turn off the gradients
    with torch.no_grad():
        # Loop through all of the validation set
        for images, labels in loader:
            if torch.cuda.is_available():
                images = images.to(device)
                #labels = torch.cuda.LongTensor(labels)
                labels = labels.float().to(device)
                
            outputs = net(images).view(-1)
            outputs[outputs > 0] = 1
            outputs[outputs < 0] = 0
            preds = outputs
            sum_accuracy += get_accuracy(labels, preds)
            sum_loss += criterion(outputs,labels)
            i += 1
    total_accuracy = sum_accuracy / i
    return (total_accuracy, sum_loss / i)

In [23]:
#torch.backends.cudnn.benchmark = True
num_epochs = 40
train_and_test(num_epochs, train_loader, dev_loader)

RuntimeError: result type Float can't be cast to the desired output type Long

In [ ]:
class SentimentNN(nn.Module):

    def __init__(self, input_dim=327):
        super(SentimentNN, self).__init__()
        self.lstm = nn.LSTM(input_dim, 200)
        self.first_layer = nn.Linear(200, input_dim)

    def forward(self, input_ids, labels=None):
        x = self.lstm(input_ids)
        x = nn.ReLU()(x)
        x = self.second_layer(x)
        x = nn.Tanh()(x)
        x = self.third_layer(x)
        return x

In [ ]:
#torch.cuda.empty_cache()

In [ ]:
import torch
from torch import nn
class Model(nn.Module):
 def __init__(self, dataset):
     super(Model, self).__init__()
     self.lstm_size = 327
     self.num_layers = 3
     self.hidden_size = 100
     self.lstm = nn.LSTM(
     input_size=self.lstm_size,
     hidden_size=self.hidden_size,
     num_layers=self.num_layers,
     dropout=0.2,
     )
     self.fc = nn.Linear(self.hidden_size, 1)
 def forward(self, x, prev_state):
         output, state = self.lstm(x, prev_state)
         logits = self.fc(output)
         return logits, state
 def init_state(self, sequence_length):
    return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
    torch.zeros(self.num_layers, sequence_length, self.lstm_size))

